# Библиотеки/Модули

In [25]:
import cv2
import tensorflow as tf
import cvlib as cv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pygetwindow
from datetime import datetime
import pyautogui
dataframe = pd.DataFrame(columns=['date', 'count_students', 'female_count', 'male_count'])

## Ячейка подсчёта студентов

In [37]:
def count_students(data):
    for folder in data:
        flagType = folder == "data/online/"
        counter = 0
        for date in os.listdir(folder):
            for image in os.listdir(folder + date + "/"):
                image = cv2.imread(folder + date + "/" + image)
                _, label, _ = cv.detect_common_objects(image, model="yolov3")
                counter+=label.count('person')
            dataframe.append({'date':date, 'typeAttendance' : flagType, 'count_students':counter, 'female_count':0, 'male_count':0}, ignore_index=True)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: C:\Users\Лавандос\.cvlib\object_detection\yolo\yolov3\yolov3.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


## Ячейка подсчёта в зависимости от пола

In [32]:
FACE_PROTO = "weights/opencv_face_detector.pbtxt"
FACE_MODEL = "weights/opencv_face_detector_uint8.pb"
GENDER_PROTO = "weights/gender_deploy.prototxt"
GENDER_MODEL = "weights/gender_net.caffemodel"

FACE_NET = cv2.dnn.readNet(FACE_MODEL, FACE_PROTO)

GENDER_NET = cv2.dnn.readNet(GENDER_MODEL, GENDER_PROTO)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
GENDER_LIST = ["Male", "Female"]

box_padding = 20
def get_face_box (net, frame, conf_threshold = 0.5):
  frame_copy = frame.copy()
  frame_height = frame_copy.shape[0]
  frame_width = frame_copy.shape[1]
  blob = cv2.dnn.blobFromImage(frame_copy, 1.0, (300, 300), [104, 117, 123], True, False)

  net.setInput(blob)
  detections = net.forward()
  boxes = []

  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > conf_threshold:
      x1 = int(detections[0, 0, i, 3] * frame_width)
      y1 = int(detections[0, 0, i, 4] * frame_height)
      x2 = int(detections[0, 0, i, 5] * frame_width)
      y2 = int(detections[0, 0, i, 6] * frame_height)
      boxes.append([x1, y1, x2, y2])
      cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height / 150)), 8)

  return frame_copy, boxes
def gender_detector (data):
  for folder in data:
    flagType = "online" in folder
    fem_count = 0
    male_count = 0
    for date in os.listdir(folder):
      for image in os.listdir(folder + date + "/"):
        image = cv2.imread(folder + "/" + image)
        resized_image = cv2.resize(image, (640, 480))
        frame = resized_image.copy()
        _, boxes = get_face_box(FACE_NET, frame)
        for box in boxes:
          face = frame[max(0, box[1] - box_padding):min(box[3] + box_padding, frame.shape[0] - 1), \
            max(0, box[0] - box_padding):min(box[2] + box_padding, frame.shape[1] - 1)]
          blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB = False)
          GENDER_NET.setInput(blob)
          gender_predictions = GENDER_NET.forward()
          gender = GENDER_LIST[gender_predictions[0].argmax()]
          if gender == "Male":
            male_count+=1
          else: fem_count+=1
      if (dataframe['date'] == date).any():
        dataframe['male_count'] = np.where(dataframe['folder'] == folder, male_count, dataframe['male_count'])
        dataframe['female_count'] = np.where(dataframe['folder'] == folder, fem_count, dataframe['female_count'])
      else: dataframe.append({'date':date, 'typeAttendance' : flagType,'count_students':fem_count + male_count, 'female_count':fem_count, 'male_count':male_count}, ignore_index=True)

  

## Графическая часть

In [28]:
def prepare_dataset():
    if "datetime" not in dataframe.dtypes:
        dataframe['date'] = pd.to_datetime(dataframe['date'], format='%d-%m-%Y')
    dataframe['count_students'] = np.where(dataframe['count_students'] < dataframe['female_count'] + dataframe['male_count'], dataframe['female_count'] + dataframe['male_count'], dataframe['count_students'])

def get_attendance_stat_avg(start_date="", end_date=""):
    prepare_dataset()
    if (start_date != "" & end_date != ""):
        mask = (dataframe['date'] > start_date) & (dataframe['date'] <= end_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость за предложенный период: ', df['count_students'].mean())
            dates = df['date']
            count_students = df['count_students']
            plt.plot(dates, count_students, linestyle = 'solid')
            plt.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (start_date != ""):
        mask = (dataframe['date'] > start_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость за предложенный период: ', df['count_students'].mean())
            dates = df['date']
            count_students = df['count_students']
            plt.plot(dates, count_students, linestyle = 'solid')
            plt.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (end_date != ""):
        mask = (dataframe['date'] < end_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость за предложенный период: ', df['count_students'].mean())
            dates = df['date']
            count_students = df['count_students']
            plt.plot(dates, count_students, linestyle = 'solid')
            plt.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    else:
        print('Средняя посещаемость за предложенный период: ', dataframe['count_students'].mean())
        dates = dataframe['date']
        count_students = dataframe['count_students']
        plt.plot(dates, count_students, linestyle = 'solid')
        plt.show()

def get_attendance_by_gender(start_date = "", end_date = ""):
    prepare_dataset()
    if (start_date != "" & end_date != ""):
        mask = (dataframe['date'] > start_date) & (dataframe['date'] <= end_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость мужчин за предложенный период: ', df['male_count'].mean())
            print('Средняя посещаемость девушек за предложенный период: ', df['female_count'].mean())
            fig, ax = plt.subplots()
            dates = df['date']
            fem = df['female_count']
            male = df['male_count']
            plt.plot(dates, fem, linestyle = 'solid', ax=ax)
            plt.plot(dates, male, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (start_date != ""):
        mask = (dataframe['date'] > start_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость мужчин за предложенный период: ', df['male_count'].mean())
            print('Средняя посещаемость девушек за предложенный период: ', df['female_count'].mean())
            fig, ax = plt.subplots()
            dates = df['date']
            fem = df['female_count']
            male = df['male_count']
            plt.plot(dates, fem, linestyle = 'solid', ax=ax)
            plt.plot(dates, male, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (end_date != ""):
        mask = (dataframe['date'] < end_date)
        try:
            df = dataframe.loc[mask]
            print('Средняя посещаемость мужчин за предложенный период: ', df['male_count'].mean())
            print('Средняя посещаемость девушек за предложенный период: ', df['female_count'].mean())
            fig, ax = plt.subplots()
            dates = df['date']
            fem = df['female_count']
            male = df['male_count']
            plt.plot(dates, fem, linestyle = 'solid', ax=ax)
            plt.plot(dates, male, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    else:
        print('Средняя посещаемость мужчин за предложенный период: ', dataframe['male_count'].mean())
        print('Средняя посещаемость девушек за предложенный период: ', dataframe['female_count'].mean())

def get_attendance_by_type(start_date = "", end_date = ""):
    prepare_dataset()
    if (start_date != "" & end_date != ""):
        mask_online = (dataframe['date'] > start_date) & (dataframe['date'] <= end_date) & (dataframe['typeAttendance' == True])
        mask_offline = (dataframe['date'] > start_date) & (dataframe['date'] <= end_date) & (dataframe['typeAttendance' == False])
        try:
            df_online = dataframe.loc[mask_online]
            df_offline = dataframe.loc[mask_offline]
            print('Средняя посещаемость онлайн за предложенный период: ', df_online['count_students'].mean())
            print('Средняя посещаемость онлайн за предложенный период: ', df_offline['count_students'].mean())
            fig, ax = plt.subplots()
            dates_online = df_online['date']
            count_online = df_online['count_students']
            dates_offline = df_offline['date']
            count_offline = df_offline['count_students']
            plt.plot(dates_online, count_online, linestyle = 'solid', ax=ax)
            plt.plot(dates_offline, count_offline, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (start_date != ""):
        mask_online = (dataframe['date'] > start_date) & (dataframe['typeAttendance' == True])
        mask_offline = (dataframe['date'] > start_date)& (dataframe['typeAttendance' == False])
        try:
            df_online = dataframe.loc[mask_online]
            df_offline = dataframe.loc[mask_offline]
            print('Средняя посещаемость онлайн за предложенный период: ', df_online['count_students'].mean())
            print('Средняя посещаемость онлайн за предложенный период: ', df_offline['count_students'].mean())
            fig, ax = plt.subplots()
            dates_online = df_online['date']
            count_online = df_online['count_students']
            dates_offline = df_offline['date']
            count_offline = df_offline['count_students']
            plt.plot(dates_online, count_online, linestyle = 'solid', ax=ax)
            plt.plot(dates_offline, count_offline, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    if (end_date != ""):
        mask_online = (dataframe['date'] <= end_date) & (dataframe['typeAttendance' == True])
        mask_offline = (dataframe['date'] <= end_date) & (dataframe['typeAttendance' == False])
        try:
            df_online = dataframe.loc[mask_online]
            df_offline = dataframe.loc[mask_offline]
            print('Средняя посещаемость онлайн за предложенный период: ', df_online['count_students'].mean())
            print('Средняя посещаемость онлайн за предложенный период: ', df_offline['count_students'].mean())
            fig, ax = plt.subplots()
            dates_online = df_online['date']
            count_online = df_online['count_students']
            dates_offline = df_offline['date']
            count_offline = df_offline['count_students']
            plt.plot(dates_online, count_online, linestyle = 'solid', ax=ax)
            plt.plot(dates_offline, count_offline, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")
    else:
        mask_online = (dataframe['typeAttendance' == True])
        mask_offline = (dataframe['typeAttendance' == False])
        try:
            df_online = dataframe.loc[mask_online]
            df_offline = dataframe.loc[mask_offline]
            print('Средняя посещаемость онлайн за предложенный период: ', df_online['count_students'].mean())
            print('Средняя посещаемость онлайн за предложенный период: ', df_offline['count_students'].mean())
            fig, ax = plt.subplots()
            dates_online = df_online['date']
            count_online = df_online['count_students']
            dates_offline = df_offline['date']
            count_offline = df_offline['count_students']
            plt.plot(dates_online, count_online, linestyle = 'solid', ax=ax)
            plt.plot(dates_offline, count_offline, linestyle = 'solid', ax=ax)
            fig.show()
        except KeyError:
            print("Указан неверный период или данных нет за данный период")

# Взаимодействие

### Переменные для работы с проектом

In [29]:
online_dir = "data/online/"
face_to_face_dir = "data/face-to-face/"
Web = True
counter_screenshots = 0
# Для случаев, когда используется Google Meet
Conf_id = "vzb-edwm-iau"
today_date = datetime.today().strftime('%d-%m-%Y')
# Даты для выбора анализируемого периода
start_date = ""
end_date = ""

### Сделать скриншот Zoom/Google Meet

In [23]:
window_titles = pygetwindow.getAllTitles()
if not os.path.exists(online_dir + today_date):
    os.makedirs(online_dir + today_date)
if Web:
    path = online_dir + today_date + f"/{str(counter_screenshots)}.png"
    matching_window = [s for s in window_titles if any(xs in s for xs in [Conf_id])]
    window = pygetwindow.getWindowsWithTitle(matching_window[0])[0]
    left, top = window.topleft
    right, bottom = window.bottomright
    pyautogui.screenshot(path)
    image = Image.open(path)
    image = image.crop((left, top, right, bottom))
    image.save(path)
    counter_screenshots+=1
else:
    path = online_dir + today_date + f"/{str(counter_screenshots)}.png"
    matching_window = [s for s in window_titles if any(xs in s for xs in ["Zoom"])]
    window = pygetwindow.getWindowsWithTitle(matching_window[0])[0]
    left, top = window.topleft
    right, bottom = window.bottomright
    pyautogui.screenshot(path)
    image = Image.open(path)
    image = image.crop((left, top, right, bottom))
    image.save(path)
    counter_screenshots+=1

### Анализ посещаемости студентов в общем (онлайн и очно)

In [34]:
count_students([online_dir, face_to_face_dir])
get_attendance_stat_avg()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: C:\Users\Лавандос\.cvlib\object_detection\yolo\yolov3\yolov4.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


### Анализ посещаемости студентов в разрезе формата занятий

### Анализ посещаемости студентов в разрезе пола 